In [1]:
import geopandas as gpd
import numpy as np
from libpysal.weights import DistanceBand
from esda.moran import Moran
from joblib import Parallel, delayed

C:\Users\DELL\AppData\Roaming\Python\Python38\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_35352\2092376092.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch sh

In [24]:
# Step 1: 读取数据
gdf = gpd.read_file(r"....shp")
gdf = gpd.read_file(r"....shp")
gdf = gdf.to_crs(epsg=32650) 

# Step 2: 准备变量
coords = np.array([(geom.x, geom.y) for geom in gdf.geometry])
values = gdf['grid_code'].values  # 替换为需要分析的字段

# Step 3: 定义函数计算某个距离下的 Moran's I
def calc_moran_i(distance):
    gdf_valid = gdf.dropna(subset=['grid_code'])
    w = DistanceBand(coords, threshold=distance, binary=True, silence_warnings=True)
    moran = Moran(values, w)
    return {
        "distance": distance,
        "I": moran.I,
        "Z": moran.z_norm,
        "p": moran.p_norm
    }

In [ ]:
# Step 4: 距离列表（单位米）
distances = list(range(10000, 30000, 500))  # 1000 到 12000 每隔1000

# Step 5: 并行计算
results = Parallel(n_jobs= 6)(delayed(calc_moran_i)(d) for d in distances)

# Step 6: 输出结果
import pandas as pd
df = pd.DataFrame(results)
df